In [3]:
from PIL import Image
from tensorflow.keras.preprocessing import image as kimage
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.models import Sequential

import numpy as np
import numpy.linalg as la


In [4]:
# Embedding model
emb_model = ResNet50(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
emb_model.trainable = False
emb_model = Sequential([emb_model, GlobalMaxPooling2D()])

In [6]:
def get_embeddings(img):
    # Normalize image
    img_array = kimage.img_to_array(img)
    expand_img = np.expand_dims(img_array, axis=0)
    preprocessed_img = preprocess_input(expand_img)

    # Run prediction
    result_to_resnet = emb_model.predict(preprocessed_img)
    flatten_result = result_to_resnet.flatten()
    result_normalized = flatten_result / la.norm(flatten_result)

    return result_normalized

ex_img = "jeans.png"
img = Image.open(ex_img)
image = img.resize((224, 224)).convert("RGB")
img_embeddings = get_embeddings(image)
img_embeddings

array([0.02651525, 0.073531  , 0.        , ..., 0.        , 0.00534359,
       0.01297403], dtype=float32)

In [27]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model

class ParallelResnet(tf.keras.Model):
    def __init__(self, *args, **kwargs):
        super(ParallelResnet, self).__init__(*args, **kwargs)
        
        # Load ResNet50 pre-trained model without top (fully connected) layers
        resnet_base = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

        # Set the base model to be non-trainable
        resnet_base.trainable = False
        
        # # Define the first sequence of the model
        # self.seq0 = tf.keras.Sequential([
        #     resnet_base.layers[1],  # conv1
        #     resnet_base.layers[2],  # bn1
        #     resnet_base.layers[3],  # relu
        #     resnet_base.layers[4],  # maxpool
        #     resnet_base.layers[5],  # layer1
        #     resnet_base.layers[6]   # layer2
        # ])
        
        # # Define the second sequence of the model
        # self.seq1 = tf.keras.Sequential([
        #     resnet_base.layers[7],  # layer3
        #     resnet_base.layers[8],  # layer4
        #     # layers.GlobalAveragePooling2D()  # avgpool
        # ])
        # Get the output of the last layer before GlobalAveragePooling2D
        last_conv_layer = resnet_base.get_layer('conv5_block3_out')
        self.seq0 = Model(inputs=resnet_base.input, outputs=last_conv_layer.output)

        # Define the second sequence of the model
        self.seq1 = tf.keras.Sequential([
            tf.keras.layers.GlobalMaxPooling2D(),
        ])
        
        # Define the fully connected layer
        # self.fc = layers.Dense(1000)  # Assuming output dimension is 1000 classes

    def predict(self, inputs):
        # apply first sequence of the model on input x
        x = self.seq0(inputs)
        # apply second sequence of the model to x
        x = self.seq1(x)    
        return x

    
def get_embeddings2(img):
    # Normalize image
    img_array = kimage.img_to_array(img)
    expand_img = np.expand_dims(img_array, axis=0)
    preprocessed_img = preprocess_input(expand_img)

    # Run prediction
    result_to_resnet = parallel_model.predict(preprocessed_img)
    print(result_to_resnet.shape)

    flatten_result = tf.keras.layers.Flatten()(result_to_resnet)
    result_normalized = flatten_result / la.norm(flatten_result)

    return result_normalized

parallel_model = ParallelResnet()
# parallel_model = Sequential([parallel_model, GlobalMaxPooling2D()])
get_embeddings2(image)

(1, 2048)


<tf.Tensor: shape=(1, 2048), dtype=float32, numpy=
array([[0.02651525, 0.073531  , 0.        , ..., 0.        , 0.00534359,
        0.01297403]], dtype=float32)>

In [21]:
resnet_base = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
print(len(resnet_base.layers))

for layer in resnet_base.layers:
    print(layer)

175
